In [1]:
import pandas as pd
import numpy as np
import ast

In [2]:
#Show all rows
#pd.set_option("max_rows", None)

TODO:
- clean csvs
    - reorder the columns for all csvs
    - split date to different columns: year/ month/ day/ hour/ minute/ second
        - for review date
        - for response date
- add column for review origin store: google/apple
- add column for dev response time
- analysis on apple dataset: short analysis for us vs ca datasets
    - add flag for countries: apple_us, apple_ca
    - then combine both

In [3]:
df_google = pd.read_csv("data/google_reviews.csv")
df_apple_us = pd.read_csv("data/apple_reviews_us.csv")
df_apple_ca = pd.read_csv("data/apple_reviews_ca.csv")

# 1.EDA

In [4]:
df_google.head()

,reviewId,userName,userImage,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,gp:AOqpTOGlaNzM8c0Ch5u_tnuIsrlJJeCWufl9tn7YRL3...,Will Gage,https://play-lh.googleusercontent.com/-e5YakL9...,I love the design of this app. It is visually ...,3,0,1.2.1,2021-04-25 12:17:31,NaN,NaN
1,gp:AOqpTOFeK_8h1wm5Xz5qsGC8YSly0qzWkCRMpB3GCNl...,austin miller,https://play-lh.googleusercontent.com/a-/AOh14...,I sent one message and was informed that I had...,1,0,1.2.1,2021-04-24 03:24:38,NaN,NaN
2,gp:AOqpTOGZn93j19ujCZW8hOOQsaa4En8z1N1ibtTPk3g...,TheClassicnathan TCn,https://play-lh.googleusercontent.com/a-/AOh14...,"""Does this app make you happy?""",5,0,1.2.1,2021-04-23 22:05:12,NaN,NaN
3,gp:AOqpTOHluwlYpU9DdvPtMobyvesxzL-AEPUGn2JeSKP...,Jeremie Mazerolle,https://play-lh.googleusercontent.com/a-/AOh14...,"Ok, the concept of this app is awesome, but is...",3,0,1.2.1,2021-04-23 04:46:57,NaN,NaN
4,gp:AOqpTOFGS5qauNo9BDUKxU6eQS7bTzdEg4QoeTldD5O...,Robert M,https://play-lh.googleusercontent.com/a-/AOh14...,Can't even try the app. Requires to validate w...,1,0,NaN,2021-04-23 02:24:40,NaN,NaN


In [5]:
df_apple_us.head()

,date,userName,isEdited,title,review,rating,developerResponse
0,2020-10-28 03:15:26,blooolp,False,Cool but empty,"I assume its just cause its new, it has great ...",4,NaN
1,2021-02-25 03:54:12,Antmo01,False,Some of my thoughts,"First off, I just wanna say that this is app i...",2,NaN
2,2020-11-05 07:41:17,jj_2077,False,"Cool concept, but poor execution and empty app","Conceptually, this is a very interesting way t...",3,NaN
3,2020-10-05 19:11:24,SoundsLikeALottaHOOPLAH,False,"A good app, but not very useful",While the idea of the app is excellent in conc...,3,NaN
4,2020-06-29 15:05:11,Xandaroth,False,Finally a legit app for gamers!?,After being burned out on all the dating apps ...,5,NaN


In [6]:
df_apple_ca.head()

,review,rating,userName,date,title,isEdited
0,Game icons don’t show when I try to add them,3,CrazyChestnut,2019-08-25 02:40:23,Bug?,False
1,Hey! There’s a bug that doesn’t allow iPhone u...,3,jesuisxiv,2021-04-01 01:37:18,Photo Card Bug,False
2,Lack of users app has to be reset a lot,1,Anthony12673454,2021-04-05 16:47:24,Lacking,False
3,"The app works alright, and has acceptable func...",3,POTATOeTREE,2020-06-24 15:46:16,"Ok, but no people.",False
4,Would really like to see this grow. But for no...,1,Xetreon,2021-03-25 12:32:47,"Good concept, no people, dead app",False


In [7]:
df_google.columns

Index(['reviewId', 'userName', 'userImage', 'content', 'score',
       'thumbsUpCount', 'reviewCreatedVersion', 'at', 'replyContent',
       'repliedAt'],
      dtype='object')

In [8]:
df_apple_us.columns

Index(['date', 'userName', 'isEdited', 'title', 'review', 'rating',
       'developerResponse'],
      dtype='object')

## (i) Data cleaning

### Dropping unwanted columns

In [9]:
def drop_col(df, cols):
    df.drop(cols, axis=1, inplace=True)

In [10]:
google_col_names_to_drop = ["reviewId", "userName", "userImage"]

In [11]:
apple_col_names_to_drop = ["userName"]

In [12]:
drop_col(df_google, google_col_names_to_drop)

In [13]:
drop_col(df_apple_us, apple_col_names_to_drop)

In [14]:
drop_col(df_apple_ca, apple_col_names_to_drop)

In [15]:
df_google.head()

,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
0,I love the design of this app. It is visually ...,3,0,1.2.1,2021-04-25 12:17:31,NaN,NaN
1,I sent one message and was informed that I had...,1,0,1.2.1,2021-04-24 03:24:38,NaN,NaN
2,"""Does this app make you happy?""",5,0,1.2.1,2021-04-23 22:05:12,NaN,NaN
3,"Ok, the concept of this app is awesome, but is...",3,0,1.2.1,2021-04-23 04:46:57,NaN,NaN
4,Can't even try the app. Requires to validate w...,1,0,NaN,2021-04-23 02:24:40,NaN,NaN


In [16]:
df_apple_us.head()

,date,isEdited,title,review,rating,developerResponse
0,2020-10-28 03:15:26,False,Cool but empty,"I assume its just cause its new, it has great ...",4,NaN
1,2021-02-25 03:54:12,False,Some of my thoughts,"First off, I just wanna say that this is app i...",2,NaN
2,2020-11-05 07:41:17,False,"Cool concept, but poor execution and empty app","Conceptually, this is a very interesting way t...",3,NaN
3,2020-10-05 19:11:24,False,"A good app, but not very useful",While the idea of the app is excellent in conc...,3,NaN
4,2020-06-29 15:05:11,False,Finally a legit app for gamers!?,After being burned out on all the dating apps ...,5,NaN


In [17]:
df_apple_ca.head()

,review,rating,date,title,isEdited
0,Game icons don’t show when I try to add them,3,2019-08-25 02:40:23,Bug?,False
1,Hey! There’s a bug that doesn’t allow iPhone u...,3,2021-04-01 01:37:18,Photo Card Bug,False
2,Lack of users app has to be reset a lot,1,2021-04-05 16:47:24,Lacking,False
3,"The app works alright, and has acceptable func...",3,2020-06-24 15:46:16,"Ok, but no people.",False
4,Would really like to see this grow. But for no...,1,2021-03-25 12:32:47,"Good concept, no people, dead app",False


In [18]:
print("apple:", "\nca:",df_apple_ca.shape, "\nus:",df_apple_us.shape)
print("google:", "\nca:",df_google.shape)

apple: 
ca: (25, 5) 
us: (266, 6)
google: 
ca: (617, 7)


### Extracting input values to proper columns

In [19]:
#How many nan in df_apple_us
df_apple_us["developerResponse"].isnull().sum()

211

In [20]:
#Show all responses that aren't nan for df_apple_us
apple_us_dev_res= pd.notnull(df_apple_us["developerResponse"])
df_apple_us[apple_us_dev_res]

,date,isEdited,title,review,rating,developerResponse
5,2020-03-12 09:00:55,False,Cool App!,"This app is so unique, i’ve never seen anythin...",4,"{'id': 14873083, 'body': 'Hello!\n\nThank you ..."
9,2019-11-15 03:59:48,True,Great App,[EDIT at bottom]\nI love the style and design ...,5,"{'id': 11831767, 'body': 'Hello! Thanks so muc..."
14,2020-06-18 03:41:33,False,Can’t do a lot,I don’t know if it’s because of screen scaling...,3,"{'id': 16261816, 'body': 'Hello,\n\nThanks for..."
18,2020-05-04 08:11:16,False,its interesting but.. 🥺,This app is pretty interesting. I like how you...,3,"{'id': 15199514, 'body': 'Hello!\n\nThank you ..."
29,2019-09-20 16:36:53,False,First Impressions,"So far, I love the app and the concept! It def...",3,"{'id': 10828984, 'body': 'Hello! We are so gla..."
30,2019-08-21 21:53:09,False,It’s Mediocre,"Although a fun and quirky app to begin with, a...",1,"{'id': 10673400, 'body': ""Hello!\n\nThank you ..."
36,2020-03-31 00:03:30,False,Great Idea! But,I love this app and the idea behind it to brin...,3,"{'id': 14872893, 'body': 'Hello!\n\nThank you ..."
40,2020-05-07 16:01:54,False,Its pretty good,I was pretty surprised by the app. Although th...,4,"{'id': 15199440, 'body': 'Hey!\n\nThanks so mu..."
47,2020-11-09 09:18:02,True,"I may reconsider at a later time, a rather dea...","I may come back to this app, it’s just not for...",2,"{'id': 14928856, 'body': 'Hello!\n\nthanks so ..."
50,2020-08-02 10:33:18,False,Deleted conversations,So I recently joined this app to see what is o...,1,"{'id': 17045920, 'body': 'Hey! We had a bug wi..."


In [21]:
#Example of "developerResponse" structure
df_apple_us["developerResponse"][5]

"{'id': 14873083, 'body': 'Hello!\\n\\nThank you so much for taking the time to leave us a review! We are so glad that you are enjoying using Kippo! We are working on adding advanced filters so you can find people based off things like console and game title. This is all great feedback and we would love to hear more. Please feel free to send us feedback or ideas anytime at sup@kippoapp.com. Please let us know if there is anything else that we can do to make Kippo a 5 star experience for you!\\n\\nBest,\\nKippo Team', 'modified': '2020-04-23T21:25:56Z'}"

In [22]:
#"developerResponse" has a dict like structure, so we need to extract them accordingly
apple_us_dev_res = df_apple_us[apple_us_dev_res]["developerResponse"].apply(lambda x: ast.literal_eval(x))

In [23]:
apple_us_dev_res = apple_us_dev_res.apply(pd.Series)

In [24]:
apple_us_dev_res

,id,body,modified
5,14873083,Hello!\n\nThank you so much for taking the tim...,2020-04-23T21:25:56Z
9,11831767,Hello! Thanks so much for being part of our co...,2019-11-14T00:25:38Z
14,16261816,"Hello,\n\nThanks for your feedback. Kippo is c...",2020-06-28T09:51:25Z
18,15199514,Hello!\n\nThank you for taking the time to lea...,2020-05-08T21:14:18Z
29,10828984,Hello! We are so glad that you love the app bu...,2019-09-23T06:42:04Z
30,10673400,Hello!\n\nThank you so much for being an early...,2019-09-13T21:55:16Z
36,14872893,Hello!\n\nThank you so much for taking the tim...,2020-04-23T21:10:07Z
40,15199440,Hey!\n\nThanks so much for taking the time to ...,2020-05-08T21:08:39Z
47,14928856,Hello!\n\nthanks so much for taking the time t...,2020-04-27T06:09:19Z
50,17045920,Hey! We had a bug with the new update that mad...,2020-08-03T10:46:13Z


In [44]:
#Clean response from \n
apple_us_dev_res["body"][5]

'Hello!\n\nThank you so much for taking the time to leave us a review! We are so glad that you are enjoying using Kippo! We are working on adding advanced filters so you can find people based off things like console and game title. This is all great feedback and we would love to hear more. Please feel free to send us feedback or ideas anytime at sup@kippoapp.com. Please let us know if there is anything else that we can do to make Kippo a 5 star experience for you!\n\nBest,\nKippo Team'

In [25]:
#Merge apple_us_dev_res back to the original df_apple_us 
df_apple_us = pd.concat([df_apple_us, apple_us_dev_res], axis=1)

In [26]:
df_apple_us.head()

,date,isEdited,title,review,rating,developerResponse,id,body,modified
0,2020-10-28 03:15:26,False,Cool but empty,"I assume its just cause its new, it has great ...",4,NaN,NaN,NaN,NaN
1,2021-02-25 03:54:12,False,Some of my thoughts,"First off, I just wanna say that this is app i...",2,NaN,NaN,NaN,NaN
2,2020-11-05 07:41:17,False,"Cool concept, but poor execution and empty app","Conceptually, this is a very interesting way t...",3,NaN,NaN,NaN,NaN
3,2020-10-05 19:11:24,False,"A good app, but not very useful",While the idea of the app is excellent in conc...,3,NaN,NaN,NaN,NaN
4,2020-06-29 15:05:11,False,Finally a legit app for gamers!?,After being burned out on all the dating apps ...,5,NaN,NaN,NaN,NaN


In [27]:
#Drop old "developerResponse" and assign "body" as the new "developerResponse"
drop_col(df_apple_us, ["developerResponse", "id"])

In [28]:
df_apple_us.rename(columns={
    "date":"revPostDate",
    "body":"developerResponse",
    "modified":"devPostDate"
}, inplace=True)

In [29]:
#Adding store and country flags
df_apple_us["store"] = "Apple"
df_apple_us["country"] = "USA"

In [30]:
df_apple_ca["store"] = "Apple"
df_apple_ca["country"] = "Canada"

In [31]:
df_apple_ca.rename(columns={"date":"revPostDate"}, inplace=True)

In [32]:
#Join both apple dataframes
df_apple = pd.concat([df_apple_us, df_apple_ca], axis=0)

In [33]:
df_apple = df_apple.reset_index(drop=True)

In [37]:
df_apple.tail()

,revPostDate,isEdited,title,review,rating,developerResponse,devPostDate,store,country
286,2019-12-29 19:17:23,False,"Decent, but not without issues","So far this seems like a great concept, implem...",4,NaN,NaN,Apple,Canada
287,2019-11-26 15:25:04,False,Need other sign-up options.,I don’t like how the only way to sign up is wi...,1,NaN,NaN,Apple,Canada
288,2019-08-16 19:49:36,False,Early but interesting,Not a lot of “dating” apps accept gaming. A lo...,5,NaN,NaN,Apple,Canada
289,2020-04-23 04:04:09,False,Need a phone number and need picture,Why it need a phone number ???\nAnd we need 4 ...,1,NaN,NaN,Apple,Canada
290,2020-03-09 13:40:29,False,Don’t think many use it,"After a month of use, I’ve just started to cli...",1,NaN,NaN,Apple,Canada


### Reordering columns

In [19]:
apple_col_names = ["title", "review", "isEdited", "rating", "developerResponse", "date"]

In [20]:
df_apple_us = df_apple_us[apple_col_names]

In [21]:
df_apple_ca["developerResponse"] = np.nan

In [22]:
df_apple_ca = df_apple_ca[apple_col_names]

In [23]:
df_apple_us.head()

,title,review,isEdited,rating,developerResponse,date
0,Cool but empty,"I assume its just cause its new, it has great ...",False,4,NaN,2020-10-28 03:15:26
1,Some of my thoughts,"First off, I just wanna say that this is app i...",False,2,NaN,2021-02-25 03:54:12
2,"Cool concept, but poor execution and empty app","Conceptually, this is a very interesting way t...",False,3,NaN,2020-11-05 07:41:17
3,"A good app, but not very useful",While the idea of the app is excellent in conc...,False,3,NaN,2020-10-05 19:11:24
4,Finally a legit app for gamers!?,After being burned out on all the dating apps ...,False,5,NaN,2020-06-29 15:05:11


In [24]:
df_apple_ca.head()

,title,review,isEdited,rating,developerResponse,date
0,Bug?,Game icons don’t show when I try to add them,False,3,NaN,2019-08-25 02:40:23
1,Photo Card Bug,Hey! There’s a bug that doesn’t allow iPhone u...,False,3,NaN,2021-04-01 01:37:18
2,Lacking,Lack of users app has to be reset a lot,False,1,NaN,2021-04-05 16:47:24
3,"Ok, but no people.","The app works alright, and has acceptable func...",False,3,NaN,2020-06-24 15:46:16
4,"Good concept, no people, dead app",Would really like to see this grow. But for no...,False,1,NaN,2021-03-25 12:32:47


In [74]:
test = pd.notnull(df_google["replyContent"])
df_google[test]

,content,score,thumbsUpCount,reviewCreatedVersion,at,replyContent,repliedAt
141,"There's a bug out there with ""please check you...",3,2,NaN,2021-02-21 20:06:08,Hey! These are some great ideas! Kippo is stil...,2020-12-04 22:13:53
185,"Love the idea and the UI of this app, even if ...",4,6,1.2.1,2021-02-05 14:13:13,"Hello, we've fixed it. Thanks for reporting :)",2021-02-05 03:44:02
474,I don't know why my review was removed but her...,1,1,1.0.23,2020-12-04 23:35:33,Hey! Thanks so much for your feedback. We've u...,2020-12-04 22:20:33
476,Good works great.,5,1,1.0.17,2020-12-04 20:27:10,Hey Audrey! Thanks so much for your 5 star rev...,2020-12-04 22:06:42
477,I waited forever for this on Android and its b...,5,0,1.0.18,2020-12-04 16:20:42,Hey Timothy! Thanks so much for your 5 star re...,2020-12-04 22:07:35
478,The fact I couldn't just used my email was a p...,1,1,NaN,2020-12-04 05:51:07,Hey Joe! I completely understand why you would...,2020-12-04 22:09:44
479,Excellent app thanks,5,1,1.0.17,2020-12-04 04:21:59,Hey Bryce! Thanks so much for your 5 star revi...,2020-12-04 22:10:18
480,It's a great Idea but it could be so much more...,3,1,1.0.23,2020-12-04 00:28:31,Hey Krissy! Thanks for your feedback! We're st...,2020-12-04 22:11:21
481,Well I like the concept but I dont like this a...,3,3,1.0.23,2020-12-03 17:00:12,Hey! Thank you for trying Kippo. We don't show...,2020-12-04 22:15:49
483,"Love it, I’ll challenge anyone to Overcooked o...",5,0,1.0.17,2020-12-03 13:47:41,Haha! I'll take you up on that challenge. Than...,2020-12-04 22:18:56


In [31]:
df_apple_ca.isnull().sum()

review      0
rating      0
date        0
title       0
isEdited    0
dtype: int64

In [ ]:
g_column_names = {
    "content": "revContent",
    "score": "stars", 
    "thumbsUpCount": "upvotesG",
    "replyContent": "devResponse"
    "repliedAt": 
}

In [18]:
df_a1["developerResponse"]

0                                                    NaN
1                                                    NaN
2                                                    NaN
3                                                    NaN
4                                                    NaN
                             ...                        
261    {'id': 16261829, 'body': 'Hello, \n\nPlease se...
262                                                  NaN
263    {'id': 15199963, 'body': 'Hello!\n\nPlease sen...
264    {'id': 14872844, 'body': "Hello!\n\nPlease sen...
265    {'id': 14872969, 'body': 'Hello,\n\nKippo is c...
Name: developerResponse, Length: 266, dtype: object

In [19]:
df_a1["developerResponse"][265]

"{'id': 14872969, 'body': 'Hello,\\n\\nKippo is currently only available for US and Canada. We will be expanding to other regions in the future!\\n\\nBest,\\nKippo Team', 'modified': '2020-04-23T21:16:29Z'}"